In [4]:
!pip install kaggle

In [6]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ningom","key":"53b8540e06ccbede8759befaff246b35"}'}

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!mv kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [11]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [12]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 5.86MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 60.6MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.5MB/s]


In [0]:
#データ取得
import pandas as pd
import numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [0]:
#drop dates
train_df.drop('PassengerId', axis=1, inplace=True)
train_df.drop('Name', axis=1, inplace=True)
train_df.drop('Ticket', axis=1, inplace=True)
train_df.drop('Cabin', axis=1, inplace=True)
#drop dates
test_df.drop('PassengerId', axis=1, inplace=True)
test_df.drop('Name', axis=1, inplace=True)
test_df.drop('Ticket', axis=1, inplace=True)
test_df.drop('Cabin', axis=1, inplace=True)

In [0]:
# 文字列の数値置換
train_df.Embarked = train_df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
train_df.Sex = train_df.Sex.replace(['male', 'female'], [0, 1])
train_df.Age = train_df.Age.replace('NaN', 0)
test_df.Embarked = test_df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
test_df.Sex = test_df.Sex.replace(['male', 'female'], [0, 1])
test_df.Age = test_df.Age.replace('NaN', 0)

In [0]:
#データの分割(t_train:正解ラベル,x_train:トレーニングデータ)
t_train=train_df["Survived"].values
x_train=train_df
x_train.drop("Survived",axis=1,inplace=True)
x_train=x_train.values

In [0]:
#データの分割
x_test=test_df.values

In [0]:
#正規化
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std_scl = MinMaxScaler() #MinMaxScaler() or Robustscaler
x_train=std_scl.fit_transform(x_train).astype(np.int64)
x_test=std_scl.fit_transform(x_test).astype(np.int64)

In [0]:
#初期値 
batch_size=32
epochs=50

In [109]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,BatchNormalization,Flatten
from keras.optimizers import Adam

model = Sequential([
    Dense(12,input_dim=7, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(8, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

history=model.fit(x_train, t_train, epochs=epochs,batch_size=batch_size)

Epoch 1/50
891/891 [==============================] - 2s 2ms/step - loss: 1.0406 - acc: 0.4714
Epoch 2/50
891/891 [==============================] - 0s 226us/step - loss: 0.9450 - acc: 0.4916
Epoch 3/50
891/891 [==============================] - 0s 229us/step - loss: 0.8391 - acc: 0.5477
Epoch 4/50
891/891 [==============================] - 0s 238us/step - loss: 0.7888 - acc: 0.5690
Epoch 5/50
891/891 [==============================] - 0s 212us/step - loss: 0.7748 - acc: 0.6004
Epoch 6/50
891/891 [==============================] - 0s 215us/step - loss: 0.7742 - acc: 0.6049
Epoch 7/50
891/891 [==============================] - 0s 211us/step - loss: 0.6666 - acc: 0.6375
Epoch 8/50
891/891 [==============================] - 0s 218us/step - loss: 0.6582 - acc: 0.6801
Epoch 9/50
891/891 [==============================] - 0s 251us/step - loss: 0.6823 - acc: 0.6476
Epoch 10/50
891/891 [==============================] - 0s 232us/step - loss: 0.6614 - acc: 0.6588
Epoch 11/50
891/891 [==========

In [110]:
#トレインデータに対するloss,acc
train_loss, train_acc = model.evaluate(x_train, t_train)
print("損失:",train_loss)
print("正確さ:",train_acc)

891/891 [==============================] - 1s 726us/step
損失: 0.5207129379046619
正確さ: 0.7744107746114635


In [0]:
#テストデータに対する予測　
predictions = np.round(model.predict(x_test))